In [2]:
from unified_planning.shortcuts import *
from unified_planning.plot import plot_sequential_plan

# Define User Types
Location = UserType("Location")
Object = UserType("Object")
Tool = UserType("Tool")

# Define Objects
tray1 = unified_planning.model.Object("tray1", Object)
tray2 = unified_planning.model.Object("tray2", Object)
mug = unified_planning.model.Object("mug", Object)
sponge = unified_planning.model.Object("sponge", Tool)
l_gripper = unified_planning.model.Object("l_gripper", Tool)
table1 = unified_planning.model.Object("table1", Location)
bin = unified_planning.model.Object("bin", Location)

# Define Fluents
on = Fluent("on", BoolType(), obj=Object, loc=Location)
holding = Fluent("holding", BoolType(), tool=Tool)
ready_to_pick = Fluent("ready_to_pick", BoolType(), obj=Object)
needs_processing = Fluent("needs_processing", BoolType(), obj=Object)  
dirty = Fluent("dirty", BoolType(), loc=Location)
clean = Fluent("clean", BoolType(), loc=Location)
wiping_allowed = Fluent("wiping_allowed", BoolType())
usable_tool = Fluent("usable_tool", BoolType(), obj=Object, tool=Tool)

# Define Actions
pick = InstantaneousAction("pick", obj=Object, loc=Location, tool=Tool)
pick_obj, pick_loc, pick_tool = pick.parameters
pick.add_precondition(on(pick_obj, pick_loc))  
pick.add_precondition(ready_to_pick(pick_obj)) 
pick.add_precondition(usable_tool(pick_obj, pick_tool)) 
pick.add_precondition(needs_processing(pick_obj))  
pick.add_effect(on(pick_obj, pick_loc), False)  
pick.add_effect(holding(pick_tool), True)  

place = InstantaneousAction("place", obj=Object, loc=Location, tool=Tool)
place_obj, place_loc, place_tool = place.parameters
place.add_precondition(holding(place_tool))  
place.add_precondition(needs_processing(place_obj))  
place.add_effect(holding(place_tool), False)  
place.add_effect(on(place_obj, place_loc), True)  
place.add_effect(needs_processing(place_obj), False)  

# Allow tray2 after tray1 is placed in the bin
allow_next_pick_tray2 = InstantaneousAction("allow_next_pick_tray2")
allow_next_pick_tray2.add_precondition(on(tray1, bin))  # Ensure tray1 is in the bin
allow_next_pick_tray2.add_precondition(needs_processing(tray2))  # Tray2 needs processing
allow_next_pick_tray2.add_effect(ready_to_pick(tray2), True)

# Allow mug after tray2 is placed in the bin
allow_next_pick_mug = InstantaneousAction("allow_next_pick_mug")
allow_next_pick_mug.add_precondition(on(tray2, bin))  # Ensure tray2 is in the bin
allow_next_pick_mug.add_precondition(needs_processing(mug))  # Mug needs processing
allow_next_pick_mug.add_effect(ready_to_pick(mug), True)

allow_wiping = InstantaneousAction("allow_wiping")
allow_wiping.add_precondition(on(tray1, bin))
allow_wiping.add_precondition(on(tray2, bin))
allow_wiping.add_precondition(on(mug, bin))
allow_wiping.add_effect(wiping_allowed, True)  

wipe = InstantaneousAction("wipe", loc=Location, tool=Tool)
wipe_loc, wipe_tool = wipe.parameters
wipe.add_precondition(dirty(wipe_loc))  
wipe.add_precondition(wiping_allowed)  
wipe.add_effect(dirty(wipe_loc), False)  
wipe.add_effect(clean(wipe_loc), True)  

# Define Problem
problem = Problem("table_cleaning")
problem.add_fluent(on, default_initial_value=False)
problem.add_fluent(holding, default_initial_value=False)
problem.add_fluent(ready_to_pick, default_initial_value=False)
problem.add_fluent(needs_processing, default_initial_value=False)
problem.add_fluent(dirty, default_initial_value=False)
problem.add_fluent(clean, default_initial_value=False)
problem.add_fluent(wiping_allowed, default_initial_value=False)
problem.add_fluent(usable_tool, default_initial_value=False)

problem.add_action(pick)
problem.add_action(place)
problem.add_action(allow_next_pick_tray2)
problem.add_action(allow_next_pick_mug)
problem.add_action(allow_wiping)
problem.add_action(wipe)

problem.add_objects([tray1, tray2, mug, sponge, l_gripper, table1, bin])

# Define Initial State
problem.set_initial_value(on(tray1, table1), True)
problem.set_initial_value(on(tray2, table1), True)
problem.set_initial_value(on(mug, table1), True)
problem.set_initial_value(dirty(table1), True)
problem.set_initial_value(holding(l_gripper), False)
problem.set_initial_value(holding(sponge), False)
problem.set_initial_value(wiping_allowed, False)
problem.set_initial_value(ready_to_pick(tray1), True)  # Start with tray1
problem.set_initial_value(ready_to_pick(tray2), False)  # Tray2 is not ready initially
problem.set_initial_value(ready_to_pick(mug), False)  # Mug is not ready initially
problem.set_initial_value(needs_processing(tray1), True)
problem.set_initial_value(needs_processing(tray2), True)
problem.set_initial_value(needs_processing(mug), True)

# Tool Constraints
problem.set_initial_value(usable_tool(tray1, l_gripper), True)
problem.set_initial_value(usable_tool(tray2, l_gripper), True)
problem.set_initial_value(usable_tool(mug, l_gripper), True)

# Define Goals
problem.add_goal(on(tray1, bin))
problem.add_goal(on(tray2, bin))
problem.add_goal(on(mug, bin))
problem.add_goal(clean(table1))

# Solve the Problem
with OneshotPlanner(name="pyperplan") as planner:
    result = planner.solve(problem)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Plan found:")
        for action in result.plan.actions:
            print(action)
        plan = result.plan
        if plan is not None:
            plot_sequential_plan(plan, figsize=(190, 195), node_size=1000000, font_size=80)
    else:
        print("No plan found.")


  *** Credits ***
  * In operation mode `OneshotPlanner` at line 118 of `/tmp/ipykernel_19935/472888635.py`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.

Plan found:
pick(tray1, table1, l_gripper)
place(tray1, bin, l_gripper)
allow_next_pick_tray2
pick(tray2, table1, l_gripper)
place(tray2, bin, l_gripper)
allow_next_pick_mug
pick(mug, table1, l_gripper)
place(mug, bin, l_gripper)
allow_wiping
wipe(table1, l_gripper)
